In [17]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df = pd.read_csv('data/data.csv', low_memory=False)

In [3]:
rename_cols = {
    '이메일 주소': 'email',
    '이메일 수신 상태': 'email_status',
    '이름': 'name',
    '구독폼 유입 경로': 'subscribe_path',
    '추천인': 'recommend_by',
    '추가 경로': 'add_path',
    '그룹': 'group',
    '광고성 정보 수신 동의': 'ad_agree_status',
    '구독일': 'subscribe_date',
    '마지막 업데이트일': 'last_update_date',
    '구독 해지 사유': 'why_unsubscribe',
    }


In [4]:
df.rename(columns=rename_cols, inplace=True)

In [5]:
df.subscribe_date = pd.to_datetime(df.subscribe_date)

In [6]:
df['sub_year'] = df.apply(lambda x: x.subscribe_date.year, axis=1)
df['sub_month'] = df.apply(lambda x: x.subscribe_date.month, axis=1)
df['sub_day'] = df.apply(lambda x: x.subscribe_date.day, axis=1)

In [9]:
months = [f'{x}월' for x in range(1,13)]

In [24]:
now_year = datetime.now().year
now_month = datetime.now().month
counts = df[df.sub_year == 2024]['sub_month'].value_counts().sort_index()
counts.index = [f'{x}월' for x in counts.index]

months = [f'{x}월' for x in range(1,13)]
counts = counts.reindex(months, fill_value=0)

# Plotly를 사용하여 바 차트 생성
fig = px.bar(
    x=counts.index,
    y=counts.values,
    labels={
        'x': f'{now_year}년 구독자',
        'y': '구독자수'
        },
    title='Monthly Distribution for the Year 2024')
fig.show()

In [32]:

# 현재 구독자 수와 목표 구독자 수 설정
current_subscribers = df[(df.sub_year == now_year) & (df.sub_month == 4)].shape[0]
goal_subscribers = 10000

# 게이지 차트 생성
fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = current_subscribers,
    domain = {'x': [0, 1], 'y': [0, 1]},
    gauge = {
        'axis': {
            'range': [None, goal_subscribers], 
            'tickwidth': 1, 
            'tickcolor': "darkblue"
            },
        'bar': {'color': "darkblue"},
        'bgcolor': "white",
        'borderwidth': 2,
        'bordercolor': "gray",
        'steps': [
            {
                'range': [0, goal_subscribers*0.5], 
                'color': 'lightgray'
            },
            {
                'range': [goal_subscribers*0.5, goal_subscribers*0.75], 
                'color': 'gray'},
            {'range': [goal_subscribers*0.75, goal_subscribers], 
                'color': 'blue'},
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': goal_subscribers*0.9}
    }
))

fig.update_layout(
    paper_bgcolor = "lavender", 
    font = {'color': "darkblue"}
)

# 차트 보기
fig.show()